In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/fraudTrain.csv',index_col=0,nrows=100000)
data=data.dropna()
data.tail()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
99995,2019-02-28 15:36:49,2475085306462014,"fraud_O'Reilly, Mohr and Purdy",home,95.14,John,Miller,M,153 Mccullough Springs Apt. 857,Lamberton,...,44.2378,-95.2739,1507,Land/geomatics surveyor,1993-10-12,085e8f89f41378f4bd21ed8a59065fee,1330443409,44.718105,-95.843397,0
99996,2019-02-28 15:37:27,4005676619255478,fraud_Kub PLC,personal_care,8.75,William,Perry,M,458 Phillips Island Apt. 768,Denham Springs,...,30.4590,-90.9027,71335,Herbalist,1994-05-31,9988dd4de1bb2182a7b3ec5f1ede1e34,1330443447,29.931844,-90.610715,0
99997,2019-02-28 15:37:34,3519232971341141,fraud_Schuppe-Schuppe,food_dining,34.20,Michael,Jones,M,754 Smith Isle,Amsterdam,...,40.4731,-80.9596,2208,Mental health nurse,1961-09-10,70151fb544afe365dc163889d67f5be6,1330443454,41.076153,-80.506107,0
99998,2019-02-28 15:38:11,4040099974063068803,fraud_Rippin-VonRueden,health_fitness,73.11,Jeffrey,Lewis,M,24255 Bryan Square,Palermo,...,48.3396,-102.2400,229,Administrator,1983-03-20,bc2adb0fad8f733cdc21ebefcc79ddf3,1330443491,48.535070,-102.524262,0
99999,2019-02-28 15:40:20,6595970453799027,fraud_Kessler Group,travel,9.08,Gina,Gomez,F,72030 Shelton Crossing Apt. 742,Stirling,...,40.6774,-74.4968,2307,Science writer,1969-09-11,fd4a6a1b893c2045d7310c70c1d9ea1f,1330443620,39.718206,-73.627417,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  100000 non-null  object 
 1   cc_num                 100000 non-null  int64  
 2   merchant               100000 non-null  object 
 3   category               100000 non-null  object 
 4   amt                    100000 non-null  float64
 5   first                  100000 non-null  object 
 6   last                   100000 non-null  object 
 7   gender                 100000 non-null  object 
 8   street                 100000 non-null  object 
 9   city                   100000 non-null  object 
 10  state                  100000 non-null  object 
 11  zip                    100000 non-null  int64  
 12  lat                    100000 non-null  float64
 13  long                   100000 non-null  float64
 14  city_pop               100000 non-null  in

In [ ]:
data.isnull().values.any()

False

In [ ]:
fraud=data[data["is_fraud"]==1]

normal=data[data["is_fraud"]==0]

print(fraud.shape,normal.shape)

(990, 22) (99010, 22)


In [ ]:
gender_mapping={"F":1,"M":0}
data["gender"]=data["gender"].map(gender_mapping)
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,1,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,1,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,0,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,0,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,0,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:
# Check the existing columns in your DataFrame
print(data.columns)

# Attempt to drop the columns, handling potential KeyError
columns_to_drop = ["merch_lat", "merch_long", "dob", "lat", "long","first","last"]
for col in columns_to_drop:
    try:
        data = data.drop(col, axis=1)
        print(f"Dropped column: {col}")
    except KeyError:
        print(f"Column not found: {col}")

data.head()

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')
Dropped column: merch_lat
Dropped column: merch_long
Dropped column: dob
Dropped column: lat
Dropped column: long
Dropped column: first
Dropped column: last


,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,city_pop,job,trans_num,unix_time,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,1,561 Perry Cove,Moravian Falls,NC,28654,3495,"Psychologist, counselling",0b242abb623afc578575680df30655b9,1325376018,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,1,43039 Riley Greens Suite 393,Orient,WA,99160,149,Special educational needs teacher,1f76529f8574734946361c461b024d99,1325376044,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,0,594 White Dale Suite 530,Malad City,ID,83252,4154,Nature conservation officer,a1a22d70485983eac12b5b88dad1cf95,1325376051,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,0,9443 Cynthia Court Apt. 038,Boulder,MT,59632,1939,Patent attorney,6b849c168bdad6f867558c3793159a81,1325376076,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,0,408 Bradley Rest,Doe Hill,VA,24433,99,Dance movement psychotherapist,a41d7549acf90789359a9aa5346dcb46,1325376186,0


In [ ]:
import re
def remove_char(text):
  return (re.sub(r'<.*?>-:', '', text))

columns=["merchant","category","job","trans_date_trans_time","street","city","state","job","trans_num"]
for col in columns:
  data[col]=data[col].apply(remove_char)

data.head()


,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,city_pop,job,trans_num,unix_time,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,1,561 Perry Cove,Moravian Falls,NC,28654,3495,"Psychologist, counselling",0b242abb623afc578575680df30655b9,1325376018,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,1,43039 Riley Greens Suite 393,Orient,WA,99160,149,Special educational needs teacher,1f76529f8574734946361c461b024d99,1325376044,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,0,594 White Dale Suite 530,Malad City,ID,83252,4154,Nature conservation officer,a1a22d70485983eac12b5b88dad1cf95,1325376051,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,0,9443 Cynthia Court Apt. 038,Boulder,MT,59632,1939,Patent attorney,6b849c168bdad6f867558c3793159a81,1325376076,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,0,408 Bradley Rest,Doe Hill,VA,24433,99,Dance movement psychotherapist,a41d7549acf90789359a9aa5346dcb46,1325376186,0


In [ ]:
columns_to_lower=["state","city","job"]
for col in columns_to_lower:
  data[col]=data[col].str.lower()

data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,city_pop,job,trans_num,unix_time,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,1,561 Perry Cove,moravian falls,nc,28654,3495,"psychologist, counselling",0b242abb623afc578575680df30655b9,1325376018,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,1,43039 Riley Greens Suite 393,orient,wa,99160,149,special educational needs teacher,1f76529f8574734946361c461b024d99,1325376044,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,0,594 White Dale Suite 530,malad city,id,83252,4154,nature conservation officer,a1a22d70485983eac12b5b88dad1cf95,1325376051,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,0,9443 Cynthia Court Apt. 038,boulder,mt,59632,1939,patent attorney,6b849c168bdad6f867558c3793159a81,1325376076,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,0,408 Bradley Rest,doe hill,va,24433,99,dance movement psychotherapist,a41d7549acf90789359a9aa5346dcb46,1325376186,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

def transform(text):
  # Check if the input is a string before applying fit_transform
  if isinstance(text, str):
    return cv.fit_transform([text]).toarray()[0] # Transform and extract single row
  else:
    return text # Return the original value if not a string

for i in columns:
  # Apply the transformation only if the column contains strings
  if data[i].dtype == 'object':
    data[i]=data[i].apply(transform)
    print(data[i])


0        [1]
1        [1]
2        [1]
3        [1]
4        [1]
        ... 
99995    [1]
99996    [1]
99997    [1]
99998    [1]
99999    [1]
Name: trans_num, Length: 100000, dtype: object


In [ ]:
data.tail()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,city_pop,job,trans_num,unix_time,is_fraud
99995,0,2475085306462014,0,0,95.14,0,0,0,0,56152,1507,0,[1],1330443409,0
99996,0,4005676619255478,0,0,8.75,0,0,0,0,70726,71335,0,[1],1330443447,0
99997,0,3519232971341141,0,0,34.20,0,0,0,0,43903,2208,0,[1],1330443454,0
99998,0,4040099974063068803,0,0,73.11,0,0,0,0,58769,229,0,[1],1330443491,0
99999,0,6595970453799027,0,0,9.08,1,0,0,0,7980,2307,0,[1],1330443620,0


In [56]:
columns_not_included=data[["cc_num","amt","gender","zip","unix_time"]]
X=data.drop('is_fraud',axis=1)
#X.columns=X.columns.astype(str)
#X[0]=X[0].fillna(data[0].mean())
X.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,city_pop,job,trans_num,unix_time
0,2,2703186189652095,0,0,4.97,1,0,0,0,28654,3495,0,[1],1325376018
1,2,630423337322,0,0,107.23,1,0,0,0,99160,149,0,[1],1325376044
2,2,38859492057661,0,1,220.11,0,0,0,0,83252,4154,0,[1],1325376051
3,1,3534093764340240,0,0,45.00,0,0,0,0,59632,1939,0,[1],1325376076
4,1,375534208663984,0,0,41.96,0,0,0,0,24433,99,0,[1],1325376186


In [60]:
!pip install imblearn



In [61]:
y=data["is_fraud"]
y.tail()

,is_fraud
99995,0
99996,0
99997,0
99998,0
99999,0


In [62]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.3,random_state=42)
Xtest.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,city_pop,job,trans_num,unix_time
75721,0,30408301059761,0,0,55.48,0,0,0,0,29436,4471,0,[1],1329199798
80184,0,4451952084362894,0,0,54.81,0,0,0,0,79063,686,0,[1],1329473396
19864,0,4666314527820883145,0,0,336.20,0,0,0,0,68869,2202,0,[1],1326421007
76699,1,377026671291680,0,0,77.63,1,0,0,0,63665,241,0,[1],1329264337
92991,0,4476840372112,0,1,3.80,0,0,0,0,35776,3395,0,[1],1330110865


In [64]:
!pip install -U imbalanced-learn  # Upgrade imblearn to the latest version

from imblearn.over_sampling import SMOTE # Import SMOTE from the correct module
smote=SMOTE()
Xtrain,ytrain=smote.fit_resample(Xtrain,ytrain)

In [68]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138624 entries, 0 to 138623
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  138624 non-null  int64  
 1   cc_num                 138624 non-null  int64  
 2   merchant               138624 non-null  int64  
 3   category               138624 non-null  int64  
 4   amt                    138624 non-null  float64
 5   gender                 138624 non-null  int64  
 6   street                 138624 non-null  int64  
 7   city                   138624 non-null  int64  
 8   state                  138624 non-null  int64  
 9   zip                    138624 non-null  int64  
 10  city_pop               138624 non-null  int64  
 11  job                    138624 non-null  int64  
 12  trans_num              138624 non-null  object 
 13  unix_time              138624 non-null  int64  
dtypes: float64(1), int64(12), object(1)


In [75]:
from sklearn.ensemble import RandomForestClassifier
model1=RandomForestClassifier(max_depth=5)
model1.fit(Xtrain,ytrain)
ypred1=model1.predict(Xtest)
print(classification_report(ytest,ypred1))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98     29698
           1       0.15      0.77      0.26       302

    accuracy                           0.95     30000
   macro avg       0.58      0.87      0.62     30000
weighted avg       0.99      0.95      0.97     30000



In [76]:
from sklearn.linear_model import LogisticRegression
model2=LogisticRegression()
model2.fit(Xtrain,ytrain)
ypred2=model2.predict(Xtest)
print(classification_report(ytest,ypred2))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     29698
           1       0.00      0.00      0.00       302

    accuracy                           0.99     30000
   macro avg       0.49      0.50      0.50     30000
weighted avg       0.98      0.99      0.98     30000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(ytest,ypred1)
print(acc)

0.9548666666666666


In [80]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(ytest,ypred2)
print(acc)

0.9899333333333333


In [82]:
pred_0=model.predict(Xtest.iloc[[0]])
print(ytest.iloc[[0]]-pred_0)

75721    0
Name: is_fraud, dtype: int64
